## Лекция 2  BM5    

query, вопрос из corpus, совпадение

первый столбец - запросы; второй столбец - коллекция документов

вопрос поступает на вход, сортируем документы, если тот с которым единичка на первом месте, все ок, если нет, то нет

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [0]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75


def bm25() -> float:
    return 

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [3]:
import os
os.chdir('C:/Users/Asus')

In [4]:
import pandas as pd

In [5]:
table = pd.read_csv('quora_question_pairs_rus.csv')

In [6]:
table.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [7]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


**Способ 1: подсчет метрики по формуле для каждой пары слово-документ**

In [89]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [154]:
inputs = []
for row in table['question1'][0:10]:
    inputs.append(str(row))

In [10]:
texts = []
for row in table['question2']:
    texts.append(str(row))

In [11]:
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.stem.PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [223]:
#tokenize documents
allWordsAllTexts = []
for text in texts[0:10000]:
    allWordsInOneText = []
    if text != "":
        for word in text.split():
            if word not in stopwords:
                word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>]', '', word).lower()
                if word != "":
                    p = morph.parse(word)[0]
                    if p not in allWordsInOneText:
                        allWordsInOneText.append(p.normal_form)
        allWordsAllTexts.append(allWordsInOneText)

In [224]:
allWordsAllTexts[9999]

['безопасно', 'женщина', 'путешествовать', 'одиночка', 'япония']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [45]:
arrayString = []
for text in allWordsAllTexts:
    s = ' '.join(text)
    arrayString.append(s)

In [227]:
arrayString[9999]

'безопасно женщина путешествовать одиночка япония'

In [47]:
X = vectorizer.fit_transform(arrayString)

In [48]:
Matrix = X.toarray()

In [49]:
lens = []
for i in arrayString:
    lens.append(len(i))

In [55]:
N = 10000 #при большем числе, программа выдаёт Memory Error

In [228]:
avgdl = sum([len(doc) for doc in allWordsAllTexts])/len(allWordsAllTexts)
avgdl

6.2999

In [50]:
import numpy as np

In [120]:
tf_matrix = Matrix/np.array(lens).reshape((-1,1))

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [108]:
tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
from math import log
words = vectorizer.get_feature_names()
num_doc_qi = np.count_nonzero(Matrix, axis=0)

In [54]:
num_doc_qi

array([1, 3, 3, ..., 1, 3, 1], dtype=int64)

In [56]:
def idf(word):
    word_id = words.index(word)
    num = num_doc_qi[word_id]
    score = log((N - num+0.5)/(num+0.5))
    return score

In [57]:
idf('кохинор')

8.804825262617976

In [73]:
all_idfs = []
for word in words:
    score = idf(word)
    all_idfs.append(score)

In [96]:
def tokenize(doc):
    allWordsInOneInput = []
    text = doc.split()
    for word in text:
        word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>]', '', word).lower()
        if word != "":
            p = morph.parse(word)[0]
            allWordsInOneInput.append(p.normal_form)
    return allWordsInOneInput

In [98]:
tokenize('очень одинокий')

['очень', 'одинокий']

In [131]:
def bm25_for_words(query, b):
    k = 2
    q_tokenized = tokenize(query)
    results = []
    for i in range(N):
        bm25 = 0
        for j in range(len(q_tokenized)):
            word = q_tokenized[j]
            if word in words:
                word_id = words.index(word)
                l = lens[i]
                tf_val = tf_matrix[(i,word_id)]
                if tf_val != 0:
                    tf_new = (tf_val*(k+1.0))/tf_val+k*(1.0-b+b*(l/avgdl))
                    bm25 += all_idfs[word_id]*tf_new
        results.append(bm25)
    return results

In [142]:
bm = bm25_for_words('сенсорный экран', 0.75)

In [143]:
bm_id = bm.index(max(bm))

In [144]:
print('Лучшее совпадение - ' + '"' + texts[bm_id] + '"')

Лучшее совпадение - "можно использовать сенсорные экраны под водой"


In [155]:
import time
start_time = time.time()
for sent in inputs:
    bm = bm25_for_words(sent, 0.75)
    bm_id = bm.index(max(bm))
    final = texts[bm_id]
print("--- %s seconds ---" % (time.time() - start_time))

--- 171.07587599754333 seconds ---


**Способ 2 (более оперативный, но не умножение векторов на матрицу)**

In [229]:
documents = {}
for idx, text in enumerate(allWordsAllTexts):
    documents[idx] = text

In [230]:
documents[9999]

['безопасно', 'женщина', 'путешествовать', 'одиночка', 'япония']

In [231]:
from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid) 

In [232]:
inverted_index['козерог']

{4, 1287}

In [233]:
import math
from math import log

In [234]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Function to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [235]:
NO_DOCS

10000

In [236]:
AVG_LEN_DOC

6.2999

In [237]:
tf_idf = create_tf_idf(2.0,0.75)

In [238]:
#Function to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Function to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in str(query).split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count] 

In [249]:
retr_docs("увеличить скорость",5)

[(7096, 12.169995561675023),
 (9880, 9.309231161049784),
 (3335, 7.8764764778490335),
 (84, 7.8764764778490335),
 (4822, 7.8764764778490335)]

In [240]:
documents[7096]

['чувствовать', 'увеличить', 'скорость', 'сердечный', 'ритм']

In [220]:
import time
start_time = time.time()
for row in inputs:
    retr_docs(row,1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00799870491027832 seconds ---


Второй способ намного оперативнее!

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

Поскольку количество документов сокращено до 10000 (чтобы не полнималась MemoryError), то для "рождественских каникул" ничего не находится:

In [250]:
retr_docs("рождественские каникулы",10)

[]

Проверим любой другой запрос:

In [251]:
retr_docs("купить телефон самсунг",10)

[(7764, 13.122248490352275),
 (6179, 11.848231413453249),
 (4987, 9.92167405714088),
 (8140, 9.175677917535253),
 (986, 9.175677917535253),
 (3746, 7.994941141142629),
 (1891, 7.940198773707717),
 (9003, 7.719724539721943),
 (9719, 7.479132940550624),
 (2856, 6.983536792996176)]

In [252]:
documents[7764]

['хороший', 'телефон', 'купить']

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1